In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException, NoSuchElementException
import time
import pandas as pd

In [6]:
ruta = r"C:\Users\9 ----- SIG\Documents\candados_resultado.xlsx"
ruta_doc = r"C:\Users\9 ----- SIG\Documents\candados_resultados.xlsx"
df = pd.read_excel(ruta)
df = df.sort_values(by = ["AÑO","SEMESTRE"], ignore_index= True)
df.head(3)

,AÑO,SEMESTRE,PERIODO,VARIABLE
0,2006,1,2006 - 1er Semestre,Estudiantes de primer curso
1,2007,1,1er Sem. de 2007,Estudiantes de primer curso
2,2008,2,2o Sem. de 2008,Inscritos - relación de inscritos


In [7]:
# Crear el navegador
driver = webdriver.Chrome()
# Abrir la página de inicio de sesión
driver.get("https://hecaa.mineducacion.gov.co/hecaa-snies/content/admin/login/login.jsf")
time.sleep(0.7)

# Ingresar usuario y contraseña
usuario = driver.find_element(By.CSS_SELECTOR, "input[id='loginForm:login-user']")
usuario.send_keys("1826-admin")
contrasena = driver.find_element(By.CSS_SELECTOR, "input[id='loginForm:login-password']")
contrasena.send_keys("1826Uan9123")

# Clic en el botón de ingresar
boton_ingresar = driver.find_element(By.ID, "loginForm:submitLogin")
boton_ingresar.click()

# Ir a la sección administrador - verificación de información
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "menu_73"))).click()
time.sleep(0.7)
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "menu_79"))).click()
time.sleep(0.7)  # Espera para que cargue completamente

# Inicializar columna de estado si no existe
if "ESTADO" not in df.columns:
    df["ESTADO"] = "Pendiente"

# Bucle para procesar cada fila del dataframe
for index, row in df.iterrows():
    print(f"\n🔄 Procesando fila {index + 1}/{len(df)}")

    periodo = str(row["PERIODO"])
    variable_buscar = str(row["VARIABLE"])

    print(f"📅 Periodo: {periodo}")
    print(f"🔍 Variable: {variable_buscar}")

    try:
        # Selección de período
        for _ in range(3):  # Reintentar hasta 3 veces
            try:
                driver.find_element(By.ID, "formFiltro:periodo").click()
                time.sleep(0.5)
                break
            except:
                time.sleep(0.5)
                continue

        # Click en la opción del periodo
        opcion = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, f"//li[@data-label='{periodo}']"))
        )
        opcion.click()
        
        # Esperar a que la tabla se cargue con el filtro aplicado
        time.sleep(0.7)  # Espera clave para el filtrado

        # BÚSQUEDA MEJORADA DE VARIABLE - SOLUCIÓN AL PROBLEMA
        encontrado = False
        celda_variable = None
        
        for intento in range(3):  # 3 intentos de búsqueda
            try:
                # Buscar en todas las celdas de la tabla
                todas_las_celdas = driver.find_elements(By.XPATH, "//table//td")
                
                for celda in todas_las_celdas:
                    if variable_buscar in celda.text:
                        celda_variable = celda
                        encontrado = True
                        print(f"✅ Variable encontrada en intento {intento + 1}")
                        break
                
                if encontrado:
                    break
                else:
                    print(f"⚠️  Variable no encontrada en intento {intento + 1}, reintentando...")
                    time.sleep(0.7)
                    
            except Exception as e:
                print(f"Error en búsqueda intento {intento + 1}: {e}")
                time.sleep(0.7)
                continue

        if not encontrado:
            df.at[index, "ESTADO"] = "Variable no encontrada"
            print("❌ Variable no encontrada después de 3 intentos")
            continue

        # Resto del proceso (esto funciona bien según tú)
        fila = celda_variable.find_element(By.XPATH, "./ancestor::tr")
        
        # Buscar el candado
        candado = fila.find_element(By.XPATH, ".//button[contains(@onclick, 'submitApertura')]")
        candado_id = candado.get_attribute('id')

        # Ejecutar JavaScript para abrir candado
        driver.execute_script(f"""
            PrimeFaces.ab({{
                s:"{candado_id}",
                f:"formDatos",
                u:"formD",
                onco:function(xhr,status,args,data){{PF('confirmDialogOpen').show();}}
            }});
        """)
        time.sleep(0.7)

        # Hacer clic en "SI" 
        try:
            boton_si = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.ID, "formDatos:j_idt125:10:j_idt162"))
            )
            boton_si.click()
        except:
            boton_si = driver.find_element(By.ID, "formDatos:j_idt125:10:j_idt162")
            driver.execute_script("arguments[0].click();", boton_si)
        
        print("✅ Clic en 'SI' ejecutado")
        time.sleep(0.5)

        # Llenar el formulario
        campo_solicitante = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.ID, "formD:nombres"))
        )
        campo_solicitante.clear()
        campo_solicitante.send_keys("Lider tecnico")

        campo_justificacion = driver.find_element(By.ID, "formD:descripcion")
        campo_justificacion.clear()
        campo_justificacion.send_keys("arreglar temas de calidad de datos")

        time.sleep(0.5)

        # Enviar formulario
        boton_enviar = driver.find_element(By.ID, "formD:submitSol")
        driver.execute_script("arguments[0].click();", boton_enviar)
        print("✅ Clic en 'Enviar' ejecutado")
        time.sleep(0.7)

        df.at[index, "ESTADO"] = "Abierto"
        print("✅ Candado abierto exitosamente")

    except Exception as e:
        df.at[index, "ESTADO"] = f"Error: {str(e)}"
        print(f"❌ Error procesando fila {index + 1}: {e}")
        
        # Recuperación simple
        try:
            driver.refresh()
            time.sleep(1)
            # Volver a los menús
            WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.ID, "menu_73"))).click()
            time.sleep(0.5)
            WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.ID, "menu_79"))).click()
            time.sleep(0.7)
        except:
            print("⚠️  No se pudo recuperar la navegación")

    # Guardar progreso
    if (index + 1) % 5 == 0:
        df.to_excel(ruta_doc, index=False)
        print(f"💾 Progreso guardado en fila {index + 1}")

# Guardar final
df.to_excel(ruta_doc, index=False)
print(f"\n✅ Proceso completado. Archivo guardado: {ruta_doc}")
print(f"📊 Resumen de estados:")
print(df["ESTADO"].value_counts())
driver.quit()


🔄 Procesando fila 1/50
📅 Periodo: 2006 - 1er Semestre
🔍 Variable: Estudiantes de primer curso
✅ Variable encontrada en intento 1
✅ Clic en 'SI' ejecutado
✅ Clic en 'Enviar' ejecutado
✅ Candado abierto exitosamente

🔄 Procesando fila 2/50
📅 Periodo: 1er Sem. de 2007
🔍 Variable: Estudiantes de primer curso
✅ Variable encontrada en intento 1
❌ Error procesando fila 2: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//button[contains(@onclick, 'submitApertura')]"}
  (Session info: chrome=144.0.7559.97); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff60fa0a535
	0x7ff60fa0a590
	0x7ff60f7b10bd
	0x7ff60f80c48e
	0x7ff60f80c79c
	0x7ff60f7fea4c
	0x7ff60f7fe8f6
	0x7ff60f859ef5
	0x7ff60f7fcc9c
	0x7ff60f7fdbe3
	0x7ff60fce7b60
	0x7ff60fce1f5d
	0x7ff60fd0311a
	0x7ff60fa260a5
	0x7ff60fa2e73c
	0x7ff60fa13